<a href="https://colab.research.google.com/github/choiminji-020102/NLP_project/blob/main/%EC%8B%A4%EC%8A%B504_230920.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Self-Attention

In [ ]:
### 필요한 라이브러리 임폴트
import tensorflow as tf
import numpy as np
import random  # 랜덤시드 고정

In [ ]:
### scaled dot product attention 함수 정의

def scaled_dot_product_attention(q, k, v):
  attention_score = tf.linalg.matmul(a=q, b=k, transpose_b=True)  # 행렬곱
  print(f'attention score : \n{attention_score}')
  print('-'*80)

  # Scaling
  dk = tf.cast(tf.shape(k)[-1], tf.float32)  # dk = 512.0
  scaled_attention_score = attention_score / tf.math.sqrt(dk)
  print(f'scale attention score : \n{scaled_attention_score}')
  print('-'*80)

  # softmax 함수 실행
  attention_weights = tf.nn.softmax(scaled_attention_score, axis=-1)

  # attention output 생성 --> attention_weights * value
  output = tf.matmul(attention_weights, v)

  return output, attention_weights

In [ ]:
### 실습용 데이터 생성

'''
1. sentence = "I love you"
2. 토큰화 --> [I, love, you]
3. 각 토큰 --> 512 차원의 임베딩 벡터로 변환  --> (3, 512) 임베딩 행렬 생성
4. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (3, 512) 임베딩 행렬에 해당하는 데이터 생성
'''

# 랜덤 시드 설정
tf.random.set_seed(0)

# (3, 512) 임베딩 행렬에 해당하는 데이터 생성
embedding_shape = (3, 512)
embedding_metrix = tf.random.normal(shape=embedding_shape, mean=0, stddev=1, seed=0)  # 정규분포로부터 랜덤한 값을 산출, 매개변수로 평균, 표준편차 지정-> 표준정규분포

# 결과 확인하기
print(f'각 단어에 대한 임베딩 행렬 생성의 결과 : \n{embedding_metrix}')

각 단어에 대한 임베딩 행렬 생성의 결과 : 
[[ 0.12391895  2.798774   -1.7961729  ... -1.2059427   0.26270926
   0.7571296 ]
 [ 1.1822435   0.14100952 -1.9312251  ...  0.13743407 -0.35198182
  -0.2877966 ]
 [ 0.5543462  -0.9617468  -0.59118867 ...  0.8685468   0.2526299
  -1.1544628 ]]


In [ ]:
### 가중치 행렬 W_q, W_k, W_v 생성
'''
1. 가중치 행렬 W_q, W_k, W_v의 모양 : (512, 512)
2. 가중치 행렬의 초기 값 --> 랜덤한 실수로 설정
3. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (512, 512) 가중치 행렬에 해당하는 데이터 W_q, W_k, W_v 생성
'''

# (512, 512*3) 모양에 해당하는 가중치 행렬 데이터 생성
weights_shape = (512, 512*3)
weights_matrix = tf.random.normal(shape=weights_shape, mean=0, stddev=1, seed=0)

# 전체 가중치 행렬의 생성 결과 확인하기
print(f'생성된 가중치 행렬 : \n{weights_matrix}')
print('-'*80)
print(f'생성된 가중치 행렬의 모양 : {weights_matrix.shape}')

print('-'*80)

# W_q, W_k, W_v 생성 결과 확인하기
W_q = weights_matrix[:, 0:512]        # 0~511
W_k = weights_matrix[:, 512:512*2]    # 512~1023
W_v = weights_matrix[:, 512*2:512*3]  # 1024~1535
print(f'가중치 행렬 W_q의 모양 : {W_q.shape}')
print('-'*80)
print(f'가중치 행렬 W_k의 모양 : {W_k.shape}')
print('-'*80)
print(f'가중치 행렬 W_v의 모양 : {W_v.shape}')

생성된 가중치 행렬의 모양 : (512, 1536)
생성된 가중치 행렬의 모양 : [[ 0.71533036  1.6461307   1.3800917  ...  1.3516908  -0.93132895
   0.7388974 ]
 [-0.27234635 -0.79476106 -1.7289692  ...  0.8388679  -0.25359774
   0.14112103]
 [-0.5283124  -0.08258526  0.73241115 ... -1.0264676   0.8124797
  -0.1822157 ]
 ...
 [ 0.6553392  -0.38194245 -0.6747198  ... -0.3752203  -0.04806583
  -1.8004427 ]
 [-1.0859361   1.2909616  -0.07064369 ... -0.12155849  0.08048517
  -0.66522104]
 [ 0.92764616  1.896207   -0.2973476  ...  0.8254814   0.07658196
   1.2026174 ]]
--------------------------------------------------------------------------------
가중치 행렬 W_q의 모양 : (512, 512)
--------------------------------------------------------------------------------
가중치 행렬 W_k의 모양 : (512, 512)
--------------------------------------------------------------------------------
가중치 행렬 W_v의 모양 : (512, 512)


In [ ]:
### q, k, v 생성

'''
임베딩 행렬과 가중치 행렬곱 --> 각 단어의 k, q, v 벡터 생성
'''

# 전체 단어의 qkv 행렬
qkv = tf.linalg.matmul(a=embedding_matrix, b=weights_matrix)

# 각 단어의 q, k, v 행렬 분해
q = qkv[:, 0:512]
k = qkv[:, 512:512*2]
v = qkv[:, 512*2:512*3]

# 결과 확인
print(f'생성된 qkv의 모양 : {qkv.shape}')
print(f'생성된 qkv의 값 : \n{qkv}')
print('-'*80)
print(f'생성된 q 행렬의 모양 : {q.shape}')
print(f'생성된 q 행렬의 값 : \n{q}')
print('-'*80)
print(f'생성된 k 행렬의 모양 : {k.shape}')
print(f'생성된 k 행렬의 값 : \n{k}')
print('-'*80)
print(f'생성된 v 행렬의 모양 : {v.shape}')
print(f'생성된 v 행렬의 값 : \n{v}')

생성된 qkv의 모양 : (3, 1536)
생성된 qkv의 값 : 
[[ -4.5343094 -12.575607  -10.582888  ...  27.454405    8.589026
    3.7817764]
 [ 20.533495    9.762379  -12.754654  ...  -7.790348    3.0119133
  -22.428392 ]
 [-13.109122  -23.546644   27.74455   ...  10.989037   -9.120724
    5.5862446]]
--------------------------------------------------------------------------------
생성된 q 행렬의 모양 : (3, 512)
생성된 q 행렬의 값 : 
[[ -4.5343094 -12.575607  -10.582888  ...   1.725666    9.89917
   53.45435  ]
 [ 20.533495    9.762379  -12.754654  ...  22.16268   -14.338249
  -26.342205 ]
 [-13.109122  -23.546644   27.74455   ...  10.530284  -24.360441
  -19.25621  ]]
--------------------------------------------------------------------------------
생성된 k 행렬의 모양 : (3, 512)
생성된 k 행렬의 값 : 
[[-18.76557    41.255527   48.42447   ...  41.221973  -16.306606
  -37.625732 ]
 [  7.694971    5.5027666  19.098648  ...  44.143616   -7.12607
    5.886017 ]
 [ 26.271389   -1.7482119 -36.467995  ...  15.689399  -13.981336
   -3.7377396]]


In [ ]:
### scaled_dot_product_attention 함수 실행
output, attention_weights = scaled_dot_product_attention(q, k, v)

# 결과 확인하기
print(f'attention_weights : \n{attention_weights}')
print('-'*80)
print(f'attention_weights 모양 : {attention_weights.shape}')
print('-'*80)
print(f'각 단어 별 새로 생성된 임베딩 벡터 : \n{output}')
print('-'*80)
print(f'각 단어 별 새로 생성된 임베딩 벡터의 모양 : {output.shape}')


attention score : 
[[16822.832  -1244.0242  -909.0869]
 [ 1226.6904 23283.93   -5462.8623]
 [ 6825.705   6555.3384 12889.601 ]]
--------------------------------------------------------------------------------
scale attention score : 
[[ 743.4712    -54.978622  -40.176346]
 [  54.21257  1029.014    -241.4267  ]
 [ 301.6564    289.70776   569.64526 ]]
--------------------------------------------------------------------------------
attention_weights : 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
--------------------------------------------------------------------------------
attention_weights 모양 : (3, 3)
--------------------------------------------------------------------------------
각 단어 별 새로 생성된 임베딩 벡터 : 
[[ -7.184684   -7.363205  -10.606537  ... -11.005928  104.29862
   11.29334  ]
 [ 37.883698  -18.500378   27.66071   ...  25.135515   12.236287
   27.44082  ]
 [ -7.512265   33.436974    7.411913  ...  35.68959    -1.2721348
   16.808563 ]]
----------------------------------------------------